In [1]:
# 0. import library
import numpy as np
import copy
from scipy.io import wavfile
from matplotlib import pyplot as plt
import pandas as pd
from sklearn import datasets
from neupy import algorithms, environment

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
c:\users\ismail\appdata\local\programs\python\python36-32\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
# 1. membuat fungsi mfcc
def Hamming_Window(n):
    w = []
    for i in range(0,n):
        temp = 0.54 - 0.46 * np.cos((2*np.pi*i)/(n-1))
        w.append(temp)
    return w

# mengambil mfcc dari suara 0-1.5 second
def getMFCC(fileDirectory):
    #get sample rate,signal,pre emphasis, dan emphasized
    sample_rateku, signalku = wavfile.read(fileDirectory)
    signalku = signalku[0:int(1.5 * sample_rateku)]
    timesku = np.arange(len(signalku))/float(sample_rateku)

    pre_emphasisku = 0.97
    emphasized_signalku = np.append(signalku[0], signalku[1:]- pre_emphasisku * signalku[:-1])
    
    # Ukuran frame adalah 25 milidetik,
    # dan overlap 15 milidetik (dalam ukuran waktu/detik)
    frame_sizeku = 0.025
    frame_strideku = 0.01

    # Konversi dari detik ke sample
    frame_lengthku = int(round(frame_sizeku * sample_rateku))
    frame_stepku = int(round(frame_strideku * sample_rateku))

    # Ukuran sinyal dalam Sample
    signal_lengthku = len(emphasized_signalku)

    # Pastikan setidaknya ada 1 frame
    num_framesku = int(np.ceil(float(np.abs(signal_lengthku - frame_lengthku)) / frame_stepku))

    pad_signal_lengthku = num_framesku * frame_stepku + frame_lengthku
    zku = np.zeros((pad_signal_lengthku - signal_lengthku))
    pad_signalku = np.append(emphasized_signalku, zku)

    indicesku = np.tile(np.arange(0, frame_lengthku), (num_framesku, 1))
    + np.tile(np.arange(0, num_framesku * frame_stepku, frame_stepku)
    , (frame_lengthku, 1)).T
    framesku = pad_signalku[indicesku.astype(np.int32, copy=False)]

    # windowing
    framesHWku = copy.copy(framesku)
    framesHWku *= np.hamming(frame_lengthku)

    # Magnitude of the FFT
    NFFTku = 512
    mag_framesku = np.absolute(np.fft.rfft(framesku, NFFTku))

    # Power Spectrum
    pow_framesku = ((1.0 / NFFTku) * ((mag_framesku) ** 2))

    # Misalkan yang diambil adalah:
    nfiltku = 40

    low_freq_melku = 0
    high_freq_melku = (2595 * np.log10(1 + (sample_rateku / 2) / 700)) # Convert Hz to Mel
    mel_pointsku = np.linspace(low_freq_melku, high_freq_melku, nfiltku + 2)
    hz_pointsku = (700 * (10**(mel_pointsku / 2595) - 1)) # Convert Mel to Hz
    bin = np.floor((NFFTku + 1) * hz_pointsku / sample_rateku)


    fbankku = np.zeros((nfiltku, int(np.floor(NFFTku / 2 + 1))))
    for m in range(1, nfiltku + 1):
        f_m_minus = int(bin[m - 1]) # left
        f_m = int(bin[m]) # center
        f_m_plus = int(bin[m + 1]) # right
        for k in range(f_m_minus, f_m):
            fbankku[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbankku[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
    filter_banksku = np.dot(pow_framesku, fbankku.T)
    filter_banksku = np.where(filter_banksku == 0, np.finfo(float).eps, filter_banksku)
    filter_banksku = 20 * np.log10(filter_banksku) # dB

    num_cepsku = 13

    from scipy.fftpack import dct
    # Keep 2-14
    mfccku = dct(filter_banksku, type=2, axis=1, norm='ortho')[:, 1 : (num_cepsku + 1)]
    mfccku -= (np.mean(mfccku, axis=0) + 1e-8)
 
    return mfccku

In [60]:
# 2. Plot semua data
#anggota = ["faruq","ismail","nurul","putra","rya"]
anggota = ["faruq","putra","rya","nurul","ismail"]
anggota_code = [1,2,3,4,5]
mfccAnggota = []
mfccAnggotaUji = []

def ambilMfcc(x):
    mfcc_row = []
    for k in range(len(x[0])):
        sumCol = 0
        for l in range(len(x)):
            #print(l,k)
            sumCol += x[l][k]
        mfcc_row.append((sumCol/(len(mfcc[k])+1)))
    return mfcc_row

for i in range(len(anggota)):
    nama = anggota[i]
    print("Memproses data:",nama)
    for j in range(0,6):
        mfcc = getMFCC("./Dataset-processed/" + nama + "/"+nama + "_" + str(j+1) + ".wav")
    for k in range(6,30):
        mfccUji = getMFCC("./Dataset-processed/" + nama + "/"+nama + "_" + str(k+1) + ".wav")
    
        mfccAnggota.append(ambilMfcc(mfcc))
        mfccAnggotaUji.append(ambilMfcc(mfccUji))

Memproses data: faruq
Memproses data: putra
Memproses data: rya
Memproses data: nurul
Memproses data: ismail


In [88]:
# 3. pengambilan dataset
environment.reproducible()
dataLatih = []
labelLatih = []
dataUji = []
labelUji = []

for i in range(len(mfccAnggota)):
    #for j in range(len(mfccAnggota[0])):
    dataLatih.append(mfccAnggota[i])
        #labelLatih.append(anggota_code[i])

for i in range(len(mfccAnggotaUji)):
    #for j in range(len(mfccAnggotaUji[0])):
    dataUji.append(mfccAnggotaUji[i])
        #labelUji.append(anggota_code[i])


In [93]:
for i in range(24):
    labelLatih.append(4)

In [83]:
print(len(labelLatih))

120


In [68]:
print(len(dataLatih))
print(len(dataUji))
#print(dataLatih == dataUji)
print(len(mfccAnggota[0]))
#print(labelLatih)

120
120
13


In [94]:
#traning data
data_train = np.vstack([dataLatih[0:18],dataLatih[24:43],dataLatih[48:67],dataLatih[72:91],dataLatih[96:115]])
target_train = np.hstack([labelLatih[0:18],labelLatih[24:43],labelLatih[48:67],labelLatih[72:91],labelLatih[96:115]])
data_test = np.vstack([dataLatih[19:23],dataLatih[44:47],dataLatih[68:71],dataLatih[72:91],dataLatih[116:119]])
target_test = np.hstack([labelLatih[19:23],labelLatih[44:47],labelLatih[68:71],labelLatih[72:91],labelLatih[116:119]])

In [95]:
lvqnet = algorithms.LVQ3(n_inputs=13,n_classes=5,verbose=True,show_epoch=20,step=0.00000001,n_updates_to_stepdrop=15*100)


Main information

[ALGORITHM] LVQ3

[OPTION] verbose = True
[OPTION] epoch_end_signal = None
[OPTION] show_epoch = 20
[OPTION] shuffle_data = False
[OPTION] train_end_signal = None
[OPTION] minstep = 1e-05
[OPTION] n_classes = 5
[OPTION] n_inputs = 13
[OPTION] n_subclasses = None
[OPTION] n_updates_to_stepdrop = 1500
[OPTION] prototypes_per_class = None
[OPTION] weight = None
[OPTION] epsilon = 0.1
[OPTION] slowdown_rate = 0.4
[OPTION] step = 1e-08



In [96]:
lvqnet.train(data_train,target_train,epochs=300)


Start training

[TRAINING DATA] shapes: (94, 13)
[TRAINING] Total epochs: 300

---------------------------------------------------------
|    Epoch    |  Train err  |  Valid err  |    Time     |
---------------------------------------------------------
|           1 |           0 |           - |       59 ms |
|          20 |           0 |           - |       32 ms |
|          40 |           0 |           - |       32 ms |
|          60 |           0 |           - |       31 ms |
|          80 |           0 |           - |       30 ms |
|         100 |           0 |           - |       30 ms |
|         120 |           0 |           - |       30 ms |
|         140 |           0 |           - |       31 ms |
|         160 |           0 |           - |       30 ms |
|         180 |           0 |           - |       32 ms |
|         200 |           0 |           - |       30 ms |
|         220 |           0 |           - |       31 ms |
|         240 |           0 |           - |       

In [97]:
prediksi = lvqnet.predict(data_test)

In [98]:
y_pred = pd.Series(prediksi, name='Prediksi')
y_actu = pd.Series(target_test, name='Aktual')
df_confusion = pd.crosstab(y_actu, y_pred)

In [99]:
df_confusion

Prediksi,0,1,2,3,4
Aktual,,,,,
0,4,0,0,0,0
1,0,3,0,0,0
2,0,0,3,0,0
3,0,0,0,19,0
4,0,0,0,0,3


In [101]:
from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(target_test, prediksi)
cm.print_stats()

Confusion Matrix:

Predicted  0  1  2   3  4  __all__
Actual                            
0          4  0  0   0  0        4
1          0  3  0   0  0        3
2          0  0  3   0  0        3
3          0  0  0  19  0       19
4          0  0  0   0  3        3
__all__    4  3  3  19  3       32


Overall Statistics:

Accuracy: 1.0
95% CI: (0.89111883932064728, nan)
No Information Rate: ToDo
P-Value [Acc > NIR]: 5.69266889176e-08
Kappa: 1.0
Mcnemar's Test P-Value: ToDo


Class Statistics:

Classes                                    0        1        2        3  \
Population                                32       32       32       32   
P: Condition positive                      4        3        3       19   
N: Condition negative                     28       29       29       13   
Test outcome positive                      4        3        3       19   
Test outcome negative                     28       29       29       13   
TP: True Positive                          4        3